In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import glob
from skimage.io import imread
import numpy as np
import pandas as pd
import h5py

In [ ]:
#! pip install tensorflow-gpu==1.10.0

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [5]:
f = h5py.File('/content/drive/MyDrive/teeth/NTP_3to5.h5','r')
a = f['data'][:]
# b = f['labels1']
c = f['labels2'][:]
f.close()

In [7]:
def combine(a,c):
  comb=[]
  for i in range(len(a)):
      two=[a[i],c[i]]
      comb.append(two)
  return comb
train_comb=combine(a,c)

In [8]:
from sklearn.model_selection import train_test_split
import random
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping , CSVLogger ,ReduceLROnPlateau ,History
from tensorflow.keras.optimizers import Adam,SGD

In [9]:
train_data, vali_data = train_test_split(train_comb,random_state=0,shuffle=True,train_size=0.7)

In [10]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.models import Model

In [11]:
def create_model():
  model = VGG16(weights='imagenet', include_top=True)
  model.layers.pop()
  x = model.layers[-1].output
  x=Dense(5, activation='relu')(x)
  x=Dense(5, activation='softmax')(x)
  # 重新建立模型結構
  model=Model(model.input,x)
  model.summary()
  return model

#以下是gpu的使用方法

In [12]:
!export GCSFUSE_REPO=gcsfuse-`lsb_release -c -s`
!echo "deb http://packages.cloud.google.com/apt $GCSFUSE_REPO main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

!sudo apt-get update
!sudo apt-get install gcsfuse

deb http://packages.cloud.google.com/apt  main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1974  100  1974    0     0  68068      0 --:--:-- --:--:-- --:--:-- 70500
OK
E: Malformed entry 1 in list file /etc/apt/sources.list.d/gcsfuse.list (Component)
E: The list of sources could not be read.
E: Malformed entry 1 in list file /etc/apt/sources.list.d/gcsfuse.list (Component)
E: The list of sources could not be read.
E: Malformed entry 1 in list file /etc/apt/sources.list.d/gcsfuse.list (Component)
E: The list of sources could not be read.


In [13]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

TPU address is grpc://10.101.84.218:8470


In [15]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#tpu=tf.distribute.TPUStrategy()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
with strategy.scope():
  model = create_model()
  model.compile( optimizer = Adam( lr = 1e-4 ) , 
              loss = 'categorical_crossentropy' , 
              metrics=['accuracy'] )

INFO:tensorflow:Initializing the TPU system: grpc://10.101.84.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.101.84.218:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

開始訓練

In [21]:
epoch = 50000000
batch_size = 4
steps_per_epoch = int( np.ceil( len( train_data )*batch_size ) )
validation_steps = int( np.ceil( len(vali_data )*batch_size ) )
model_path =   '/content/drive/MyDrive/teeth/saved_models/NTP-3to5.h5' 
check_point = ModelCheckpoint( model_path,monitor='val_loss' , 
                              save_best_only= True , verbose = 1)
csv_logger = CSVLogger( '/content/drive/MyDrive/teeth/logs/NTP-3to5.log '  )
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, 
#                   patience=40, min_delta=0.0001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=50, min_lr=0.0001)

In [17]:
x_train=[]
y_train=[]
x_vail=[]
y_vail=[]
for i in range(len(train_data)):
  x_train.append(train_data[i][0])
  y_train.append(train_data[i][1])
x_train=np.stack(x_train)
y_train=np.stack(y_train)
for i in range(len(vali_data)):
  x_vail.append(vali_data[i][0])
  y_vail.append(vali_data[i][1])
x_vail=np.stack(x_vail)
y_vail=np.stack(y_vail)

In [23]:
model.fit( x=(x_train),y=(y_train), 
          epochs=epoch , 
          batch_size=40,
          verbose=True , 
          steps_per_epoch=steps_per_epoch  , 
          callbacks=[ check_point , csv_logger , reduce_lr ] , 
          validation_data=((x_vail),(y_vail)),
          validation_steps = validation_steps)

Epoch 1/50000000
   2/2468 [..............................] - ETA: 2:09 - loss: 1.5777 - accuracy: 0.2625WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0038s vs `on_train_batch_end` time: 0.0987s). Check your callbacks.


2468/2468 [==============================] - ETA: 0s - loss: 1.5092 - accuracy: 0.3290WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0018s vs `on_test_batch_end` time: 0.0180s). Check your callbacks.



Epoch 00001: val_loss improved from inf to 1.53415, saving model to /content/drive/MyDrive/teeth/saved_models/NTP-3to5.h5
2468/2468 [==============================] - 279s 113ms/step - loss: 1.5092 - accuracy: 0.3290 - val_loss: 1.5341 - val_accuracy: 0.3208
Epoch 2/50000000
2468/2468 [==============================] - ETA: 0s - loss: 1.5079 - accuracy: 0.3290
Epoch 00002: val_loss improved from 1.53415 to 1.53397, saving model to /content/drive/MyDrive/teeth/saved_models/NTP-3to5.h5
2468/2468 [==============================] - 274s 111ms/step - loss: 1.5079 - accuracy: 0.3290 - val_loss: 1.5340 - val_accuracy: 0.3208
Epoch 3/50000000
2468/2468 [==============================] - ETA: 0s - loss: 1.5077 - accuracy: 0.3291
Epoch 00003: val_loss improved from 1.53397 to 1.53377, saving model to /content/drive/MyDrive/teeth/saved_models/NTP-3to5.h5
2468/2468 [==============================] - 274s 111ms/step - loss: 1.5077 - accuracy: 0.3291 - val_loss: 1.5338 - val_accuracy: 0.3208
Epoch 

In [ ]:
#result = model.fit_generator( data_gen , epochs=epoch , verbose=True , 
#                              steps_per_epoch=steps_per_epoch  , 
#                              callbacks=[ check_point , csv_logger , reduce_lr ] , 
#                              validation_data = val_data,validation_steps = validation_steps)

In [ ]:
#! pip install tensorflow-gpu==2.0.0